In [1]:
import re
import numpy as np
import pandas as pd

In [5]:
awd = pd.read_csv('awards.csv.gz')
acc = pd.read_csv('accused.csv.gz')
ase = pd.read_csv('ase-units.csv.gz')

/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def list_diff(list1, list2):
    '''returns list after taking set difference of two lists'''
    return list(set(list1) - set(list2))

def intersect(list1, list2):
    '''returns list after taking set intersection of two lists'''
    return list(set(list1) & set(list2))

In [4]:
name_cols = ['full_name', 'first_name', 'last_name', 'middle_initial', 'middle_name', 'suffix_name']
acc = acc[intersect(acc.columns, name_cols)]
acc.head()

""
0
1
2
3
4


In [202]:
def split_full_names(names, ln='Last.Name', fn='First.Name'):
    '''returns pandas dataframe of last and first name columns
       made from splitting input pandas series of full names
    '''
    # Fill NaN names with ',' for ease of use
    names = names.fillna(",")
    # Fill names that do not contain letter ('-') as ',' for ease of use
    names = names.map(lambda x: x if re.search('[a-zA-Z]', x) else ",")
    # Split names after last comma into list of length = 2
    # Ex: 'Jones, JR, Bob' -> ['Jones, JR', 'Bob']
    names = names.map(lambda x: x.rsplit(',', 1))
    # Turn series of lists into dataframe
    # with last and first name columns
    names = pd.DataFrame(names.values.tolist(),
                         columns=[ln, fn])
    return names

In [203]:
df = acc.copy()
df_cols = acc.columns
if 'Full.Name' in df_cols:
        # Split full names into last and first names
        name_df = split_full_names(df['Full.Name'])
    # Otherwise (generally first and last names)
else:
    name_df = df.fillna("")  # Fill NAs with empty strings

In [183]:
def extract_suffix_name(x, suffixes):
    '''returns the suffix name in a given string'''
    # Split input name (x) by spaces and look for suffix in name pieces
    suffix = [w for w in x.split(" ") if w in suffixes]
    # If any suffix found, return it, else return empty string
    return suffix[0] if suffix else ""


def extract_middle_initial(x, mi_pattern, suffixes):
    '''returns the middle initial in a given string'''
    # Search input name for middle initial based on regex pattern
    mi = re.search(mi_pattern, x)
    # Assign identified middle initial if it is found
    # And if the name is too short (Ex: A J, J R)
    # Otherwise mi is assigned as an empty string
    mi = mi.group() if mi and len(x) > 3 else ''
    
    # If middle initial is not empty
    # And name is not DE LA _ (middle initial is not valid)
    # And the identified middle initial is not in the suffixes
    if (mi and
        not (re.search('^DE LA\s[A-Z]$', x) or
             mi[1] in suffixes)):
        # Remove the middle initial from the name
        x = x.replace(mi, ' ')
        # Remove spaces in middle initial
        mi = mi.replace(' ', '')
    else:
        # Assign middle initial as empty string
        mi = ''
    return x, mi

In [185]:
#name_df[name_df['First.Name'].str.contains('  ') | name_df['Last.Name'].str.contains('MAC')]

In [186]:
def full_strip_name(x):
    '''returns string after
       removing any redundant whitespace or punctuation from string
    '''
    x = re.sub(r'[^\w\s]', '', x)
    return ' '.join(x.split())

def basic_strip_name(x):
    '''returns string after
       removing any redundant whitespace or period from string
    '''
    x = re.sub('\s\s+', ' ',
               re.sub(r'^\s|\.|\s$', '', x))
    return x


In [190]:
def split_name(x, mi_pattern, suffixes):
    # Extract middle initial
    x, mi = extract_middle_initial(x, mi_pattern, suffixes)
    x, mi2 = extract_middle_initial(x, mi_pattern, suffixes)
    # Extract suffix name
    suff = extract_suffix_name(x, suffixes)
    # If suffix is not empty
    if suff:
        # Remove suffix from name
        x = re.sub('(^{0}\s)|(\s{0}$)'.format(suff),
                   '', x)
        # Remove white space from suffix
        suff = suff.replace(' ', '')
    # If name is empty after suffix removed
    # EX: first name = JR
    if not x:
        x = suff    # Re-assign suffix as name
        suff = ''   # Replace suffix with empty string

    # Return list of cleaned name, middle initial, and suffix
    return [full_strip_name(x).replace(' ', ''), mi, mi2, suff, basic_strip_name(x)]

In [191]:
def clean_name_col(col):
    # Store name of input column
    col_name = col.name
    # Remove redundant spaces and periods
    col = col.map(basic_strip_name)
    # Ensure string is in uppercase
    col = col.map(str.upper)

    # If col is First or Last Name
    if col_name in ['First.Name', 'Last.Name']:
        # Choose regex pattern for middle initial depending on column:
        # If First Name: single letters starting, ending, or middle of string
        # If Last Name: single letters ending or middle of string
        # EX: This avoids O Brien or D Angelo mistakes
        mi_pattern = ('(^[A-Z]\s)|(\s[A-Z])\s|\s[A-Z]$'
                      if col_name == 'First.Name'
                      else '(\s[A-Z])\s|\s[A-Z]$')
        # Chooses suffixes to look for depending on column:
        # First Name columns probably do no contain V as a suffix
        # V may be mistaken as a middle initial
        # But Last Name columns might contain V or I as a suffix
        suffixes = ['II', 'III', 'IV', 'JR', 'SR']
        if col_name == 'Last.Name':
            suffixes.extend(['V', 'I'])
        # Return pandas dataframe created from list of lists
        # Comprised of first/last name, middle initial, suffix name
        # middle initial and suffix name elements are abbrivated
        # And which column they came from is idenified by F/L
        return pd.DataFrame([split_name(x, mi_pattern, suffixes)
                             for x in col],
                            columns=[col_name,
                                     col_name[0] + '_MI',
                                     col_name[0] + '_MI2',
                                     col_name[0] + '_SN',
                                     col_name + '_NS'])

    # Else if column is a middle name column
    elif col_name == 'Middle.Name':
        # Return pandas dataframe of middle name
        # And first letter of middle name (middle initial)
        # If there is not middle name, return empty strings
        return pd.DataFrame([['', '']
                             if not (isinstance(x, str) and x)
                             else [x, x[0]]
                             for x in col],
                            columns=[col_name, 'Middle.Initial'])

    # If column is middle initial
    elif col_name == 'Middle.Initial':
        # Return middle initial column
        return col

    # If column not recognized, raise error
    else:
        raise Exception('Uh.. what sort of names?')

In [192]:
# Split first name column into first name, middle initial, suffix
fn_df = clean_name_col(name_df['First.Name'])
# Split last name column into last name, middle initial, suffix
ln_df = clean_name_col(name_df['Last.Name'])

In [193]:
ln_df.iloc[117690,]

Last.Name        MCCORMACK
L_MI                      
L_MI2                     
L_SN                      
Last.Name_NS    MC CORMACK
Name: 117690, dtype: object

In [194]:
# Join name dataframes
cleaned_df = fn_df.join(ln_df)

In [195]:
def compare_middle_initials(mi_df):
    '''returns pandas dataframe and list of conflicts'''
    output_list = []    # Initialize output list
    conflict_indexes = []   # Initialize conflict index list
    # Iterate over the the middle initial columns
    # In order of importance any given middle initial column takes priority
    # (or it is totally empty), then come middle initials found in first names
    # And lastly middle initials found in last names (very rare)
    for i, v, w, x, y, z in mi_df[['Middle.Initial', 'F_MI', 'F_MI2', 'L_MI', 'F_MI']].itertuples():
        # Initialize (unique) set of middle initials
        mis = set((v, w, x, y, z))
        # Remove all empty strings
        mis.discard('')
        # If the set is empty (all middle initials were empty)
        if not mis:
            # Add tuple of two  empty strings to output
            output_list.append(('', ''))
        # If there is only 1 real middle initial in columns
        elif len(mis) == 1:
            # Add first (only) middle initial and empty string to output
            output_list.append((mis.pop(), ''))
        # If there are two real middle initials
        elif len(mis) == 2:
            # Append both (since the values are in order of importance)
            output_list.append((mis))
        # Else (if there are 3 unique values)
        else:
            # Add this index to the conflict list
            conflict_indexes.append(i)
            # Add first and second middle initials to output
            output_list.append(tuple(mis)[:2])

    # Return dataframe of first (primary) middle initials
    # and secondary middle initials
    # and list of conflict indexes
    return (pd.DataFrame(output_list,
                         columns=['Middle.Initial',
                                  'Middle.Initial2']),
            conflict_indexes)


def compare_suffix_names(sn_df):
    '''returns pandas dataframe and list of conflicts'''
    output_list = []    # Initialize output list
    conflict_indexes = []   # Initialize conflict index list
    # Iterate over the the suffix name columns
    # In order of importance any given suffix name column takes priority
    # (or it is totally empty), then come suffix names found in last names
    # And lastly suffix names found in first names (rare)
    for i, x, y, z in sn_df[['Suffix.Name', 'L_SN', 'F_SN']].itertuples():
        # Initialize (unique) set of suffix names
        sns = set((x, y, z))
        # Remove all empty strings
        sns.discard('')
        # If the set is empty (all suffix names were empty)
        if not sns:
            # Add empty string to output
            output_list.append('')
        # If there is only 1 real suffix name in columns
        elif len(sns) == 1:
            output_list.append(sns.pop())
        # If there are more than 1 unique suffixes
        else:
            # Add index to conflicts list
            conflict_indexes.append(i)
            # Append first element to output
            output_list.append(sns.pop())
    # Return pandas dataframe of 1 Suffix Name column
    # and list of conflict indexes
    return (pd.DataFrame(output_list,
                         columns=['Suffix.Name']),
            conflict_indexes)

In [196]:
# Join name dataframes
cleaned_df = fn_df.join(ln_df)

# If suffix not in columns already (almost always)
if 'Suffix.Name' not in df_cols:
    # Insert suffix name column of empty strings
    cleaned_df.insert(0, 'Suffix.Name', '')

# If middle name is already in columns
if 'Middle.Name' in df_cols:
    # Join cleaned middle name and initial columns to cleaned_df
    cleaned_df = cleaned_df.join(clean_name_col(name_df['Middle.Name']))
# If middle initial is already in columns
elif 'Middle.Initial' in df_cols:
    # Join cleaned middle initial column to cleaned_df
    cleaned_df = cleaned_df.join(clean_name_col(name_df['Middle.Initial']))
# If no middle inital columns in original dataframe
else:
    # Insert middle initial column of empty strings
    cleaned_df.insert(0, 'Middle.Initial', '')

# Create middle initial dataframe (Middle.Initial, Middle.Initial2)
# And collect indexes of conflicting middle initials
mi_df, mi_conflicts = compare_middle_initials(cleaned_df[['Middle.Initial',
                                                          'F_MI','F_MI2', 'L_MI', 'L_MI2']])
# Create suffix name dataframe (Suffix.Name)
# And collect indexes of conflicting suffix names
sn_df, sn_conflicts = compare_suffix_names(cleaned_df[['Suffix.Name',
                                                       'F_SN', 'L_SN']])

# Identify name columns which exist in cleaned_df
name_cols = intersect(cleaned_df.columns,
                      ['First.Name', 'Last.Name', 'Middle.Name','Last.Name_NS', 'First.Name_NS'])
# Remove middle initial and suffix columns from cleaned dataframe
# Then joined cleaned dataframe to middle initial and suffix name dfs
cleaned_df = cleaned_df[name_cols].join(mi_df).join(sn_df)

In [197]:
cleaned_df[cleaned_df['First.Name']!=cleaned_df['First.Name_NS']]

,First.Name,Last.Name,Last.Name_NS,First.Name_NS,Middle.Initial,Middle.Initial2,Suffix.Name
216,KIMTOY,HUH,HUH,KIM-TOY,,,
604,MARYANN,PASKULY,PASKULY,MARY ANN,,,
3462,DEANDRE,REDD,REDD,DE ANDRE,,,
4036,CORASUE,ROBERTS,ROBERTS,CORA SUE,,,
4381,DEARMAND,CARTER,CARTER,DE ARMAND,,,
4800,MARYANN,PASKULY,PASKULY,MARY ANN,,,
5852,DEARMAND,CARTER,CARTER,DE ARMAND,,,
6484,ROSEANN,PHILLIPS,PHILLIPS,ROSE ANN,,,
6558,ROSEANN,PHILLIPS,PHILLIPS,ROSE ANN,,,
6667,MARYANN,PASKULY,PASKULY,MARY ANN,,,


In [198]:
cleaned_df.loc[115229,]

First.Name          ERIN
Last.Name          MARSH
Last.Name_NS       MARSH
First.Name_NS       ERIN
Middle.Initial         M
Middle.Initial2        E
Suffix.Name             
Name: 115229, dtype: object

In [199]:
def clean_names(df):
    '''returns pandas dataframe of cleaned name columns'''
    df_cols = df.columns.values  # Store column names
    # If names are Full.Names
    if 'Full.Name' in df_cols:
        # Split full names into last and first names
        name_df = split_full_names(df['Full.Name'])
    # Otherwise (generally first and last names)
    else:
        name_df = df.fillna("")  # Fill NAs with empty strings

    # Split first name column into first name, middle initial, suffix
    fn_df = clean_name_col(name_df['First.Name'])
    # Split last name column into last name, middle initial, suffix
    ln_df = clean_name_col(name_df['Last.Name'])
    # Join name dataframes
    cleaned_df = fn_df.join(ln_df)

    # If suffix not in columns already (almost always)
    if 'Suffix.Name' not in df_cols:
        # Insert suffix name column of empty strings
        cleaned_df.insert(0, 'Suffix.Name', '')

    # If middle name is already in columns
    if 'Middle.Name' in df_cols:
        # Join cleaned middle name and initial columns to cleaned_df
        cleaned_df = cleaned_df.join(clean_name_col(name_df['Middle.Name']))
    # If middle initial is already in columns
    elif 'Middle.Initial' in df_cols:
        # Join cleaned middle initial column to cleaned_df
        cleaned_df = cleaned_df.join(clean_name_col(name_df['Middle.Initial']))
    # If no middle inital columns in original dataframe
    else:
        # Insert middle initial column of empty strings
        cleaned_df.insert(0, 'Middle.Initial', '')
    # Create middle initial dataframe (Middle.Initial, Middle.Initial2)
    # And collect indexes of conflicting middle initials
    mi_df, mi_conflicts = compare_middle_initials(cleaned_df[['Middle.Initial',
                                                              'F_MI','F_MI2', 'L_MI', 'L_MI2']])
    # Create suffix name dataframe (Suffix.Name)
    # And collect indexes of conflicting suffix names
    sn_df, sn_conflicts = compare_suffix_names(cleaned_df[['Suffix.Name',
                                                           'F_SN', 'L_SN']])

    # Identify name columns which exist in cleaned_df
    name_cols = intersect(cleaned_df.columns,
                          ['First.Name', 'Last.Name', 'Middle.Name','Last.Name_NS', 'First.Name_NS'])
    # Remove middle initial and suffix columns from cleaned dataframe
    # Then joined cleaned dataframe to middle initial and suffix name dfs
    cleaned_df = cleaned_df[name_cols].join(mi_df).join(sn_df)

    # Return tuple of cleaned names dataframe and conflict indexes
    return (cleaned_df, mi_conflicts + sn_conflicts)

In [201]:
cdf, confl = clean_names(name_df)

In [206]:
confl

[]

In [207]:
cdf.head(40)

,First.Name,Last.Name,Last.Name_NS,First.Name_NS,Middle.Initial,Middle.Initial2,Suffix.Name
0,WILLIAM,BARRON,BARRON,WILLIAM,,,
1,KIMBERLY,C0NNOLLY,C0NNOLLY,KIMBERLY,,,
2,JOHN,KEENE,KEENE,JOHN,,,
3,SCOTT,SLAVIN,SLAVIN,SCOTT,,,
4,ANTONIO,MARTINEZ,MARTINEZ,ANTONIO,,,
5,MICHAEL,JONES,JONES,MICHAEL,,,
6,CORNELIUS,BROWN,BROWN,CORNELIUS,,,
7,KENNETH,MOLESKY,MOLESKY,KENNETH,,,
8,SOFIA,TERRONES,TERRONES,SOFIA,,,
9,JAMES,BRILL,BRILL,JAMES,,,


In [212]:
df = awd.copy()
df_cols = awd.columns

In [213]:
cdf, confl = clean_names(name_df)

In [214]:
cdf[cdf['First.Name']!=cdf['First.Name_NS']].head(40)

,First.Name,Last.Name,Last.Name_NS,First.Name_NS,Middle.Initial,Middle.Initial2,Suffix.Name
216,KIMTOY,HUH,HUH,KIM-TOY,,,
604,MARYANN,PASKULY,PASKULY,MARY ANN,,,
3462,DEANDRE,REDD,REDD,DE ANDRE,,,
4036,CORASUE,ROBERTS,ROBERTS,CORA SUE,,,
4381,DEARMAND,CARTER,CARTER,DE ARMAND,,,
4800,MARYANN,PASKULY,PASKULY,MARY ANN,,,
5852,DEARMAND,CARTER,CARTER,DE ARMAND,,,
6484,ROSEANN,PHILLIPS,PHILLIPS,ROSE ANN,,,
6558,ROSEANN,PHILLIPS,PHILLIPS,ROSE ANN,,,
6667,MARYANN,PASKULY,PASKULY,MARY ANN,,,


In [216]:
cdf[cdf['Last.Name']!=cdf['Last.Name_NS']].head(40)

,First.Name,Last.Name,Last.Name_NS,First.Name_NS,Middle.Initial,Middle.Initial2,Suffix.Name
37,JOSEPH,MCCARTHY,MC CARTHY,JOSEPH,,,
57,JOSEPH,MCGUIRE,MC GUIRE,JOSEPH,,,
109,PATRICK,MCAULIFFE,MC AULIFFE,PATRICK,,,
144,MARTIN,MCNAUGHTON,MC NAUGHTON,MARTIN,,,
151,KEVIN,OBRIEN,O BRIEN,KEVIN,,,
157,LISA,ROCCOPIGNATO,ROCCO-PIGNATO,LISA,,,
160,JUDITH,HILTDYSON,HILT-DYSON,JUDITH,,,
179,PAUL,MCGRAW,MC GRAW,PAUL,,,
267,THOMAS,VANHOVE,VAN HOVE,THOMAS,,,
286,MARK,MCNAMARA,MC NAMARA,MARK,,,


In [215]:
cdf[cdf['Middle.Initial']!=cdf['Middle.Initial2']].head(40)

,First.Name,Last.Name,Last.Name_NS,First.Name_NS,Middle.Initial,Middle.Initial2,Suffix.Name
2021,TIMOTHY,OBRIEN,O BRIEN,TIMOTHY,P,,
6860,TIMOTHY,OBRIEN,O BRIEN,TIMOTHY,P,,
11527,TIMOTHY,OBRIEN,O BRIEN,TIMOTHY,P,,
12468,TIMOTHY,OBRIEN,O BRIEN,TIMOTHY,P,,
13790,RICHARD,GRAND,GRAND,RICHARD,J,,
14201,TIMOTHY,OBRIEN,O BRIEN,TIMOTHY,P,,
18267,ERIN,MARSH,MARSH,ERIN,M,E,
18367,JUAN,CAUINIAN,CAUINIAN,JUAN,A,G,
22668,JUAN,CAUINIAN,CAUINIAN,JUAN,A,G,
24348,TIMOTHY,OBRIEN,O BRIEN,TIMOTHY,P,,
